<a href="https://colab.research.google.com/github/VictoryBeforeFight/KOSA_ML_Project/blob/main/1_0_%EC%96%B8%EB%8D%94_%EB%9D%BC%EB%B2%A8_%EB%9E%9C%ED%8F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/머신러닝 프로젝트/광고 클릭/train.parquet')

In [ ]:
#결측치 제거
train.drop(columns=['ID', 'F11', 'F27', 'F29'], inplace=True)

In [ ]:
#결측치 처리
for column in train.columns:
    if train[column].dtype == 'object':
        train[column].fillna(-1, inplace=True)
    else:
        train[column].fillna(train[column].median(), inplace=True)

In [ ]:
categorical_columns = train.select_dtypes(include=['object']).columns

#타입변경
train[categorical_columns] = train[categorical_columns].astype(str)

#범주형 데이터 인코딩
for col in categorical_columns:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])

In [ ]:
# 목표 변수와 특성 분리
X = train.drop('Click', axis=1)
y = train['Click']

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
y = (y > y.median()).astype(int)
# 언더샘플링
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
# 모델 초기화 및 훈련
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# 예측
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
# 성능 평가 보고서 생성
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
from sklearn.metrics import roc_auc_score
#AUC 계산
y_true = y_test  # 실제 타깃 값
y_pred = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_true, y_pred)
print(f'AUC: {auc:.4f}')